# Walkthru 12
> walkthru 12
- toc: true 
- badges: true
- comments: true
- categories: [course22]
- image: images/paddy.png

In [56]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0"
from fastai.vision.all import *
from fastcore.parallel import *
path = Path()
trn_path = path/'train_images'

In [57]:
arch ='convnext_tiny_in22k'

In [58]:
df = pd.read_csv(path/'train.csv')
df

,image_id,label,variety,age
0,100330.jpg,bacterial_leaf_blight,ADT45,45
1,100365.jpg,bacterial_leaf_blight,ADT45,45
2,100382.jpg,bacterial_leaf_blight,ADT45,45
3,100632.jpg,bacterial_leaf_blight,ADT45,45
4,101918.jpg,bacterial_leaf_blight,ADT45,45
...,...,...,...,...
10402,107607.jpg,tungro,Zonal,55
10403,107811.jpg,tungro,Zonal,55
10404,108547.jpg,tungro,Zonal,55
10405,110245.jpg,tungro,Zonal,55


In [3]:
dls = ImageDataLoaders.from_folder(
    trn_path, valid_pct=.2, seed=42, item_tfms=Resize(224),batch=aug_transforms(size=224, min_scale= 0.75),bs=32)   

In [4]:
learn = vision_learner(dls, arch, metrics=error_rate)

In [5]:
learn.fine_tune(1, 0.02)

epoch,train_loss,valid_loss,error_rate,time
0,1.392303,0.888152,0.271985,00:38


epoch,train_loss,valid_loss,error_rate,time
0,0.507126,0.295987,0.097069,01:55


### Model and the summary

In [6]:
m = learn.model

In [6]:
m

Sequential(
  (0): TimmBody(
    (model): ConvNeXt(
      (stem): Sequential(
        (0): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
        (1): LayerNorm2d((96,), eps=1e-06, elementwise_affine=True)
      )
      (stages): Sequential(
        (0): ConvNeXtStage(
          (downsample): Identity()
          (blocks): Sequential(
            (0): ConvNeXtBlock(
              (conv_dw): Conv2d(96, 96, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=96)
              (norm): LayerNorm((96,), eps=1e-06, elementwise_affine=True)
              (mlp): Mlp(
                (fc1): Linear(in_features=96, out_features=384, bias=True)
                (act): GELU()
                (drop1): Dropout(p=0.0, inplace=False)
                (fc2): Linear(in_features=384, out_features=96, bias=True)
                (drop2): Dropout(p=0.0, inplace=False)
              )
              (drop_path): Identity()
            )
            (1): ConvNeXtBlock(
              (conv_dw): Conv2d(96, 

In [21]:
learn.summary()

Sequential (Input shape: 32 x 3 x 224 x 224)
Layer (type)         Output Shape         Param #    Trainable 
                     32 x 96 x 56 x 56   
Conv2d                                    4704       True      
LayerNorm2d                               192        True      
Identity                                                       
Conv2d                                    4800       True      
LayerNorm                                 192        True      
____________________________________________________________________________
                     32 x 56 x 56 x 384  
Linear                                    37248      True      
GELU                                                           
Dropout                                                        
____________________________________________________________________________
                     32 x 56 x 56 x 96   
Linear                                    36960      True      
Dropout                            

### What is in it?

In [7]:
h= m[1]
h

Sequential(
  (0): AdaptiveConcatPool2d(
    (ap): AdaptiveAvgPool2d(output_size=1)
    (mp): AdaptiveMaxPool2d(output_size=1)
  )
  (1): Flatten(full=False)
  (2): BatchNorm1d(1536, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (3): Dropout(p=0.25, inplace=False)
  (4): Linear(in_features=1536, out_features=512, bias=False)
  (5): ReLU(inplace=True)
  (6): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (7): Dropout(p=0.5, inplace=False)
  (8): Linear(in_features=512, out_features=10, bias=False)
)

In [10]:
ll =h[-1]

In [11]:
ll

Linear(in_features=512, out_features=10, bias=False)

In [12]:
ll.parameters()

<generator object Module.parameters at 0x7f3eb07f6510>

In [13]:
llp=list(ll.parameters())[0]

In [14]:
llp

Parameter containing:
tensor([[-0.0744,  0.0336,  0.0605,  ..., -0.0420,  0.0424, -0.0034],
        [-0.1795, -0.2076,  0.1650,  ..., -0.0621, -0.0006,  0.0082],
        [ 0.1246,  0.1805, -0.0646,  ...,  0.0427, -0.0914, -0.0676],
        ...,
        [-0.0006,  0.0500,  0.1109,  ..., -0.1318, -0.1138,  0.0212],
        [-0.0090,  0.0812, -0.0604,  ...,  0.1045,  0.2306,  0.1239],
        [-0.0352, -0.0684,  0.1508,  ...,  0.0299,  0.0247, -0.0539]],
       device='cuda:0', requires_grad=True)

In [15]:
llp.shape

torch.Size([10, 512])

In [34]:
ll

Linear(in_features=512, out_features=10, bias=False)

In [59]:
from copy import deepcopy

In [60]:
dls = ImageDataLoaders.from_folder(trn_path, valid_pct=.2, seed=42, item_tfms=Resize(224),batch=aug_transforms(size=224, min_scale= 0.75))   

In [80]:
learn = vision_learner(dls, arch)#, metrics=error_rate) # If you miss this part, you'll get an error! metrics=error_rate creates a problem 

In [72]:
learn2 = deepcopy(learn)

In [73]:
curr_loss=learn2.loss_func

In [74]:
def dtc_loss(preds,targs):
    rice_preds,dis_preds = preds    
    return curr_loss(dis_preds,targs)
    

In [75]:
class DiseaseAndTypeClassifier(nn.Module):
    def __init__(self,m):
        super().__init__()
        self.l1 = nn.Linear(in_features=512, out_features=10, bias=False) #rice type
        self.l2 = nn.Linear(in_features=512, out_features=10, bias=False) #disease
        del(m[1][-1]) #it removes the last layer
        self.m = m
    def forward (self,x):
        x = self.m(x)
        x1 = self.l1(x)
        x2 = self.l2(x)
        return x1,x2
        

In [76]:
dtc = DiseaseAndTypeClassifier(learn2.model)

In [77]:
learn2.model = dtc

In [78]:
learn2.loss_func=dtc_loss

In [79]:
preds,targs = learn2.get_preds(dl=learn2.dls.valid)

In [54]:
rice_preds, dis_preds = preds

In [55]:
rice_preds.shape,dis_preds.shape

(torch.Size([2081, 10]), torch.Size([2081, 10]))

**need to check Fastai create head function**